## Concepts of MapReduce in Dask

### What is MapReduce?

MapReduce is a programming model and an associated implementation for processing and generating big data sets with a parallel, distributed algorithm on a cluster. [Wikipedia](https://en.wikipedia.org/wiki/MapReduce)



A MapReduce program is composed of a map procedure (or method), which performs filtering, sorting or element elaborations (such as sorting out students by first name, one queue for each name), and a reduce method, which performs a summary operation (such as counting the number of students in each queue, yielding name frequencies). The MapReduce  frameowork it's essential when you have big data problems and a cluster.
It's realiable and fault tolerant and allows to work on huge data (Exabyte) on a cluster without memory issues.

The model is a specialization version of the ```split-apply-combine``` approache for data analysis. Each action of the map or reduce function is made on a partition of data with an indipendet process.
Each worker of a cluster execute the same operations on separated chunks of data. Once the computation is terminated the results are retrieved from each worker node and then they are combined toghter. If necessary the map-reduce function are applied again on the results combination. This process is repeated until all the operations are done and all the results are merge into single one which is the real final results.


Let's see an example of the Map function. Try to convert the ```CRSDepTime``` column to a timestamp. In the dataset the CRSDepTime which is a timestamps as HHMM, it is stored as integers in the csv:

In [ ]:
import os
import dask
import dask.dataframe as dd

df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': str,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})



crs_dep_time = df.CRSDepTime.head(10)
crs_dep_time

In order to convert these to timestamps of scheduled departure time, we need to convert these integers into ```pd.Timedelta``` objects, and then combine them with the ```Date``` column.

In [ ]:
%time
import pandas as pd

# Get the first 10 dates to complement our `crs_dep_time`
date = df.Date.head(10)

# Get hours as an integer, convert to a timedelta
hours = crs_dep_time // 100
hours_timedelta = pd.to_timedelta(hours, unit='h')

# Get minutes as an integer, convert to a timedelta
minutes = crs_dep_time % 100
minutes_timedelta = pd.to_timedelta(minutes, unit='m')

# Apply the timedeltas to offset the dates by the departure time
departure_timestamp = date + hours_timedelta + minutes_timedelta
departure_timestamp

Let's try with the MapReduce.
Dask.dataframe provides a few methods to make applying custom functions to Dask DataFrames easier:

+ map_partitions
+ map_overlap
+ reduction
Here we'll just be discussing map_partitions, which we can use to implement to_timedelta on our own:

In [ ]:
%time 
hours = df.CRSDepTime // 100
# hours_timedelta = pd.to_timedelta(hours, unit='h')
hours_timedelta = hours.map_partitions(pd.to_timedelta, unit='h')

minutes = df.CRSDepTime % 100
# minutes_timedelta = pd.to_timedelta(minutes, unit='m')
minutes_timedelta = minutes.map_partitions(pd.to_timedelta, unit='m')

departure_timestamp = df.Date + hours_timedelta + minutes_timedelta

In [ ]:
departure_timestamp

In [ ]:
departure_timestamp.head()

### Exercise1: 
Try to rewrite the above code to use a single call to map_partitions.

#### Hints:
All code of the function will be executed on chunks/partition of data

In [ ]:
def compute_departure_timestamp(df):
    #write your code here 


    return df


In [ ]:
departure_timestamp = df.map_partitions(compute_departure_timestamp)


In [ ]:
departure_timestamp.head()